# GBTランカーモデルの作り方

一般的な考え方は、各セッション（つまりユーザー）に対して、50～200の候補（正しい予測である可能性が高い）を見つけ、GBTランカーモデルを学習して最終的に20を選択する、というものです。

そのためには、ランカーモデル用の学習データを作成する必要があります。

# 候補者データフレーム

ランカーモデルの学習データを作成するための最初のステップは、候補のデータフレームを作成することです。

高品質な候補を生成する一つの方法として、co-visitiation matricesを使用する方法があります。

候補データフレームは、1行に1組のセッション（＝ユーザ）とエイド（＝アイテム）を持ちます。このデータフレームは以下の列を持ちます。

* session (i.e. user)
* aid (i.e. item)
* user features
* item features
* user-item interaction features
* click target (i.e 0 or 1)
* cart target (i.e. 0 or 1)
* order target (i.e. 0 or 1)

# メモリ管理

最新バージョンのcuDF（バージョン22.08以降）をローカルで使用する場合、デフォルトのビット幅を32に設定する新しい機能があります。

これにより、データフレームがint64やfloat64を使用することがなくなり、使い勝手が良くなります。
```
import cudf

cudf.set_option("default_integer_bitwidth", 32)

cudf.set_option("default_float_bitwidth", 32)
```
カラムのdtypesを減らしても、メモリに問題がある場合があります。

これを解決する方法は、すべてをチャンクに分割して処理することです。

train.groupby('session')を用いてユーザ特徴量を作成する場合、まず、訓練データをX個（2、4、8個など）に分割することが考えられます。

そして、各個別にユーザ特徴量を処理し、f'user_features_p{PIECE_NUMBER}.pqt'としてディスクに保存してください。

後で読み込むときに、それらを連結することができます。


# スピード - GPUの使用

このデータには約1300万人のユーザーと200万点のアイテムがあるため、以下のコードは全て実行に時間がかかるでしょう。

以下の処理には、NvidiaのRAPIDS cuDFのように、CPUの代わりにGPUを使用して高速化されたデータフレームライブラリを使用することをお勧めします。

# ステップ1

トレーニングデータは、Kaggleトレーニングの最初の3週間分のデータを使用します。

そして、検証データはKaggle trainの最後の1週間分とします。


検証データA（つまりKaggle trainの最後の1週間）のすべてのユーザー（つまりセッション）に対して、X個の候補aidを生成します。

ここでは、X=50とする。ここで、(number_of_session x 50, 2 )という形状のデータフレームを作成します。

各セッションは50回ずつ登場します。

また、[session,aid]のペアが重複することはないでしょう。

検証データのユーザーしかターゲットにしないので、検証データのユーザーしか使いません（以下のステップ6で）。

# ステップ2

アイテム特徴を作成する。

訓練データ＋検証データA（テストリークを使用）を用いて、項目特徴を独自のデータフレームで作成し、ディスクにパーケットを保存します。

例えば
```
item_features = train.groupby('aid').agg({'aid':'count','session':'nunique','type':'mean'})

item_features.columns = ['item_item_count','item_user_count','item_buy_ratio']

CONVERT COLUMNS TO INT32 and FLOAT32 HERE
item_features.to_parquet('item_features.pqt')
```
メモリに問題がある場合はご注意ください。

train を 10 個の dataframe part に分割する．

train_part_1.groupby('aid') が正しく動作するように、1つの項目に関連するすべての行は、同じデータフレーム部分でなければなりません。

処理後、各パートを別々にディスクに保存する。そして、後でディスクから読み込む際に、候補データフレームにマージする前に、それらを連結する。

# ステップ3

ユーザー特徴量の作成 

検証データAを用いて、ユーザー特徴を独自のデータフレームに作成し、ディスクにパーケットを保存します。

例えば

```
user_features = train.groupby('session').agg({'session':'count','aid':'nunique','type':'mean'})
user_features.columns = ['user_user_count','user_item_count','user_buy_ratio']
CONVERT COLUMNS TO INT32 and FLOAT32 HERE
user_features.to_parquet('user_features.pqt')
```

# ステップ4

このステップはオプションです。

ステップ4はCVとLBを改善しますが、GBTランカーはステップ4なしでも動作します。

ユーザーアイテムインタラクション特徴量の作成 

検証データAを用いて、複数のユーザーアイテム特徴量データフレームを作成し、ディスクにパーケットとして保存する。

各アイデアに対して、新しいデータフレームを作成することができる。

1つのデータフレームには、ユーザがクリックしたすべての項目を格納することができます。

そこで、1列user、1列item、3列目item_clickedを持つデータフレームを作成します。

そして、ユーザがクリックしたユニークなアイテムごとに、item_clicked = 1の新しい行を追加します。このデータフレームには、['user','item']のペアの重複行がないことに注意してください。

このデータフレームをディスクに保存します。これを候補データフレームにマージする際には、クリックされなかった項目を示すために、fillna(0)を使用します。

# ステップ5

候補データフレームに特徴量を追加する。

候補データフレームに特徴を追加するために、以下のようにディスクから読み込んでマージします。



```
item_features = pd.read_parquet('item_features.pqt')
candidates = candidates.merge(item_features, left_on='aid', right_index=True, how='left').fillna(-1)
user_features = pd.read_parquet('user_features.pqt')
candidates = candidates.merge(user_features, left_on='session', right_index=True, how='left').fillna(-1)
```

注：もしすべての特徴を候補データフレームにマージする際にメモリに問題がある場合は、これを分割して行うことができます。

```
CHUNKS = 10
chunk_size = np.ceil( len(candidates) / CHUNKS)
for k in range(CHUNKS):
    df = candidates.iloc[k*chunk_size:(k+1)*chunk_size].copy()
    df = df.merge(item_features, left_on='aid', right_index=True, how='left').fillna(-1)
    df = df.merge(user_features, left_on='session', right_index=True, how='left').fillna(-1)
    df.to_parquet(f'candidate_with_features_p{k}.pqt')
```



# ステップ6

ステップ1で作成した候補データフレームにターゲットを追加します。

targetの列を追加する最も良い方法は、dataframe mergeを使用することです。

まず、以下のようにすべてのtarget=1のdataframeを作成します。

以下のようなリストの列としてターゲットを含むdataframeからスタートします（ここではRadekのground truth labelsのように）。

```
tar = pd.read_parquet('test_labels.parquet')
tar = tar.loc[ tar['type']=='carts' ]
aids = tar.ground_truth.explode().astype('int32').rename('item')
tar = tar[['session']].astype('int32').rename({'session':'user'},axis=1)
tar = tar.merge(aids, left_index=True, right_index=True, how='left')
tar['cart'] = 1
```

# トレーニング

GBT ランカーモデル用の学習データができました。

GroupKFoldを使って学習する必要があります。

重要：学習時には、user と item のカラムは特徴量として使用せず、他のカラムのみを使用します。
```
FEATURES = candidates.columns[2 : -1*len(targets)]
```
 とします。XGBでは、objectiveパラメータを変更することで、rank:pairwise, rank:ndcg, rank:mapの3種類のランカーを選択することができることに注意してください。(XGBのパラメータ 'max_depth', 'subsample', 'colsample_bytree', 'learning_rate' も追加して調整する必要があります）。

```
import xgboost as xgb
from sklearn.model_selection import GroupKFold

skf = GroupKFold(n_splits=5)
for fold,(train_idx, valid_idx) in enumerate(skf.split(candidates, candidates['click'], groups=candidates['user'] )):

    X_train = candidates.loc[train_idx, FEATURES]
    y_train = candidates.loc[train_idx, 'click']
    X_valid = candidates.loc[valid_idx, FEATURES]
    y_valid = candidates.loc[valid_idx, 'click']

    # IF YOU HAVE 50 CANDIDATE WE USE 50 BELOW
    dtrain = xgb.DMatrix(X_train, y_train, group=[50] * (len(train_idx)//50) ) 
    dvalid = xgb.DMatrix(X_valid, y_valid, group=[50] * (len(valid_idx)//50) ) 

    xgb_parms = {'objective':'rank:pairwise', 'tree_method':'gpu_hist'}
    model = xgb.train(xgb_parms, 
        dtrain=dtrain,
        evals=[(dtrain,'train'),(dvalid,'valid')],
        num_boost_round=1000,
        verbose_eval=100)
    model.save_model(f'XGB_fold{fold}_click.xgb')
```

注: GPU で XGB をトレーニングする際にメモリに問題がある場 合、frac = 0.5, 0.25, 0.1, または 0.05 でネガティブを 2x, 4x, 10x, 20x にダウンサンプ リングすることを検討してください（その後 DMatrix でグループサイズを 更新してください）。または、複数の GPU で DASK XGB を使用します。以下はコード例です。

```
positives = candidates.loc[candidates['click']==1]
negatives = candidates.loc[candidates['click']==0].sample(frac=0.5)
candidates = pd.concat([positives,negatives],axis=0,ignore_index=True)
```

# 推論
推論では、Kaggleのテストデータから新しい候補データフレームを作成します（以前の候補生成のテクニックを使用）。

そして、4週間分のKaggle trainと1週間分のKaggle testからitemの特徴量を作成します。

そして、Kaggleのテストデータからユーザー特徴を作成します。

これらの特徴量を候補にマージします。そして、保存されたモデルを使って、クリックの予測を推測します。

最後に、予測結果をソートして、20個を選びます。
```
preds = np.zeros(len(test_candidates))
for fold in range(5):
    model = xgb.Booster()
    model.load_model(f'XGB_fold{fold}_click.xgb')
    model.set_param({'predictor': 'gpu_predictor'})
    dtest = xgb.DMatrix(data=test_candidates[FEATURES])
    preds += model.predict(dtest)/5
predictions = test_candidates[['user','item']].copy()
predictions['pred'] = preds

predictions = predictions.sort_values(['user','pred'], ascending=[True,False]).reset_index(drop=True)
predictions['n'] = predictions.groupby('user').item.cumcount().astype('int8')
predictions = predictions.loc[predictions.n<20]
sub = predictions.groupby('user').item.apply(list)
sub = sub.to_frame().reset_index()
sub.item = sub.item.apply(lambda x: " ".join(map(str,x)))
sub.columns = ['session_type','labels']
sub.session_type = sub.session_type.astype('str')+ '_clicks'
```

メモリエラーが発生した場合の注意 テストデータの1/10をロードすることを検討してください。

その後、特徴をマージする。次に推論を行う。

次の1/10をロードし、特徴量をマージし、推論を行う。最後に予測結果を連結してsubmission.csvを作る

# validation

In [ ]:
VER = 1
FEATURES = [
        'user', 'item_item_count', 'item_user_count', 
        'item_buy_ratio', 'user_user_count', 'user_item_count',
        'user_buy_ratio']

In [ ]:
!nvidia-smi

Mon Jan 23 14:03:04 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  A100-SXM4-40GB      Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    51W / 400W |      0MiB / 40536MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd, numpy as np
import pickle, glob, gc

from collections import Counter
import itertools
# multiprocessing 
import psutil
from multiprocessing import Pool
from sklearn.model_selection import GroupKFold
import psutil
N_CORES = psutil.cpu_count()     # Available CPU cores
print(f"N Cores : {N_CORES}")
from multiprocessing import Pool

N Cores : 12


In [ ]:
def merge_candidate(SVER,IVER,UVER,TYPE,MODE):
    candidates = pd.read_parquet(f'/content/drive/MyDrive/Colab Notebooks/kaggle/OTTO/dataset/candidate/suggest/{TYPE}/{MODE}_{TYPE}{SVER}.pqt')
    candidates['session'] = candidates.index
    candidates = candidates.set_index('session')
    item_features = pd.read_parquet(f'/content/drive/MyDrive/Colab Notebooks/kaggle/OTTO/dataset/candidate/item/{MODE}_item{IVER}.pqt')
    candidates = candidates.merge(item_features, left_on='item', right_index=True, how='left').fillna(-1)
    user_features = pd.read_parquet(f'/content/drive/MyDrive/Colab Notebooks/kaggle/OTTO/dataset/candidate/user/{MODE}_user{UVER}.pqt')
    candidates = candidates.merge(user_features, left_on='session', right_index=True, how='left').fillna(-1)
    candidates['user'] = candidates.index
    candidates = candidates.set_index('user')
    return candidates

In [ ]:
def merge_target(TYPE,candidates):
    tar = pd.read_parquet('/content/drive/MyDrive/Colab Notebooks/kaggle/OTTO/dataset/otto-validation/test_labels.parquet')
    tar = tar.loc[ tar['type']==TYPE ]
    aids = tar.ground_truth.explode().astype('int32').rename('item')
    tar = tar[['session']].astype('int32').rename({'session':'user'},axis=1)
    tar = tar.merge(aids, left_index=True, right_index=True, how='left')
    tar[TYPE] = 1
    candidates = candidates.merge(tar,on=['user','item'],how='left').fillna(0)
    return candidates

In [ ]:
!pip install -q xgboost==1.6.2
import xgboost as xgb
from sklearn.model_selection import GroupKFold
from sklearn.metrics import recall_score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.9/255.9 MB 6.0 MB/s eta 0:00:00


In [ ]:
def train_xgb(candidates,TARGET):
    preds = np.zeros(len(candidates))
    skf = GroupKFold(n_splits=5)
    for fold,(train_idx, valid_idx) in enumerate(skf.split(candidates, candidates[TARGET], groups=candidates['user'] )):

        X_train = candidates.loc[train_idx, FEATURES]
        y_train = candidates.loc[train_idx, TARGET]
        X_valid = candidates.loc[valid_idx, FEATURES]
        y_valid = candidates.loc[valid_idx, TARGET]

        X_train = X_train.sort_values("user").reset_index(drop=True)
        X_valid = X_valid.sort_values("user").reset_index(drop=True)

        train_group = X_train.groupby('user').user.agg('count').values
        valid_group = X_valid.groupby('user').user.agg('count').values

        X_train = X_train.drop(["user"], axis=1)
        X_valid = X_valid.drop(["user"], axis=1)

        dtrain = xgb.DMatrix(X_train, y_train,group=train_group)
        dvalid = xgb.DMatrix(X_valid, y_valid,group=valid_group)

        xgb_parms = {
            'objective':'rank:pairwise', 
            'tree_method':'gpu_hist',
            'random_state': 42, 
            'learning_rate': 0.1,
            "colsample_bytree": 0.8, 
            'max_depth': 6,
        }
        model = xgb.train(xgb_parms, 
            dtrain=dtrain,
            evals=[(dtrain,'train'),(dvalid,'valid')],
            num_boost_round=1000,
            verbose_eval=500)
        preds[valid_idx] = model.predict(dvalid)
        model.save_model(f'XGB_fold{fold}_{TARGET}.xgb')
    predictions = candidates[['user','item']].copy()
    predictions['pred'] = preds
    predictions = predictions.sort_values(['user','pred'], ascending=[True,False]).reset_index(drop=True)
    predictions['n'] = predictions.groupby('user').item.cumcount().astype('int8')
    predictions = predictions.loc[predictions.n<20]
    sub = predictions.groupby('user').item.apply(list)
    sub = sub.to_frame().reset_index()
    sub.item = sub.item.apply(lambda x: " ".join(map(str,x)))
    sub.columns = ['session','labels']
    sub.labels = sub.labels.apply(lambda x: [int(i) for i in x.split(' ')[:20]])
    test_labels = pd.read_parquet('/content/drive/MyDrive/Colab Notebooks/kaggle/OTTO/dataset/otto-validation/test_labels.parquet')
    test_labels = test_labels.loc[test_labels['type']==TARGET]
    test_labels = test_labels.merge(sub, how='left', on=['session'])
    test_labels['hits'] = test_labels.apply(lambda df: len(set(df.ground_truth).intersection(set(df.labels))), axis=1)
    test_labels['gt_count'] = test_labels.ground_truth.str.len().clip(0,20)
    recall = test_labels['hits'].sum() / test_labels['gt_count'].sum()
    print('{} Recall = {:.5f}'.format(TARGET,recall))

## clicks

In [ ]:
%%time
candidates = merge_candidate(VER,'clicks','val')
candidates = merge_target('clicks',candidates)
train_xgb(candidates,'clicks')
del candidates
_ = gc.collect()

[0]	train-map:0.65015	valid-map:0.64898
[500]	train-map:0.59663	valid-map:0.59213
[999]	train-map:0.59999	valid-map:0.59269
[0]	train-map:0.65552	valid-map:0.65602
[500]	train-map:0.59669	valid-map:0.59167
[999]	train-map:0.60006	valid-map:0.59251
[0]	train-map:0.66121	valid-map:0.65993
[500]	train-map:0.59723	valid-map:0.59186
[999]	train-map:0.60055	valid-map:0.59253
[0]	train-map:0.66388	valid-map:0.66355
[500]	train-map:0.59709	valid-map:0.59281
[999]	train-map:0.60027	valid-map:0.59334
[0]	train-map:0.66999	valid-map:0.66960
[500]	train-map:0.59716	valid-map:0.59261
[999]	train-map:0.60065	valid-map:0.59303
clicks Recall = 0.52558


## carts

In [ ]:
%%time
candidates = merge_candidate(VER,'carts','val')
candidates = merge_target('carts',candidates)
train_xgb(candidates,'carts')
del candidates
_ = gc.collect()

[0]	train-map:0.91989	valid-map:0.91948
[500]	train-map:0.91648	valid-map:0.91404
[999]	train-map:0.91749	valid-map:0.91403
[0]	train-map:0.92038	valid-map:0.91995
[500]	train-map:0.91635	valid-map:0.91371
[999]	train-map:0.91745	valid-map:0.91364
[0]	train-map:0.92242	valid-map:0.92168
[500]	train-map:0.91662	valid-map:0.91316
[999]	train-map:0.91758	valid-map:0.91312
[0]	train-map:0.92305	valid-map:0.92282
[500]	train-map:0.91646	valid-map:0.91412
[999]	train-map:0.91754	valid-map:0.91424
[0]	train-map:0.92204	valid-map:0.92258
[500]	train-map:0.91624	valid-map:0.91479
[999]	train-map:0.91726	valid-map:0.91487
carts Recall = 0.40965


## orders

In [ ]:
%%time
candidates = merge_candidate(VER,'orders','val')
candidates = merge_target('orders',candidates)
train_xgb(candidates,'orders')
del candidates
_ = gc.collect()

[0]	train-map:0.95522	valid-map:0.95512
[500]	train-map:0.94850	valid-map:0.94647
[999]	train-map:0.94950	valid-map:0.94647
[0]	train-map:0.96078	valid-map:0.95986
[500]	train-map:0.94873	valid-map:0.94557
[999]	train-map:0.94971	valid-map:0.94559
[0]	train-map:0.95866	valid-map:0.95804
[500]	train-map:0.94864	valid-map:0.94599
[999]	train-map:0.94964	valid-map:0.94596
[0]	train-map:0.96273	valid-map:0.96277
[500]	train-map:0.94853	valid-map:0.94625
[999]	train-map:0.94958	valid-map:0.94624
[0]	train-map:0.96110	valid-map:0.96169
[500]	train-map:0.94838	valid-map:0.94689
[999]	train-map:0.94937	valid-map:0.94686
orders Recall = 0.64924


# inference

In [ ]:
def predict(test_candidates,TYPE):
    preds = np.zeros(len(test_candidates))
    test_candidates.reset_index(inplace=True)
    for fold in range(5):
        model = xgb.Booster()
        model.load_model(f'XGB_fold{fold}_{TYPE}.xgb')
        model.set_param({'predictor': 'gpu_predictor'})
        dtest = xgb.DMatrix(data=test_candidates[FEATURES].drop(["user"], axis=1))
        preds += model.predict(dtest)/5
    predictions = test_candidates[['user','item']].copy()
    predictions['pred'] = preds
    predictions = predictions.sort_values(['user','pred'], ascending=[True,False]).reset_index(drop=True)
    predictions['n'] = predictions.groupby('user').item.cumcount().astype('int8')
    predictions = predictions.loc[predictions.n<20]
    sub = predictions.groupby('user').item.apply(list)
    sub = sub.to_frame().reset_index()
    sub.item = sub.item.apply(lambda x: " ".join(map(str,x)))
    sub.columns = ['session_type','labels']
    sub.session_type = sub.session_type.astype('str')+ f'_{TYPE}'
    return sub

## clicks

In [ ]:
%%time
test_candidates = merge_candidate(VER,'clicks','test')
clicks_pred_df = predict(test_candidates,'clicks')
del test_candidates
_ = gc.collect()

## carts

In [ ]:
%%time
test_candidates = merge_candidate(VER,'carts','test')
carts_pred_df = predict(test_candidates,'carts')
del test_candidates
_ = gc.collect()

## orders

In [ ]:
%%time
test_candidates = merge_candidate(VER,'orders','test')
orders_pred_df = predict(test_candidates,'orders')
del test_candidates
_ = gc.collect()

# submission

In [ ]:
pred_df = pd.concat([clicks_pred_df, orders_pred_df, carts_pred_df])
pred_df.columns = ["session_type", "labels"]
pred_df.to_csv(f"xgb{VER}.csv", index=False)
pred_df.to_csv(f'/content/drive/MyDrive/Colab Notebooks/kaggle/OTTO/submission/xgb{VER}.csv', index=False)

In [ ]:
!pip install kaggle -q
import os
import json
f = open("/content/drive/MyDrive/Colab Notebooks/kaggle/kaggle.json", 'r')
json_data = json.load(f)
os.environ['KAGGLE_USERNAME'] = json_data['username']
os.environ['KAGGLE_KEY'] = json_data['key']

In [ ]:
!kaggle competitions submit -c otto-recommender-system -f xgb1.csv -m ""

100% 780M/780M [00:19<00:00, 43.0MB/s]
Successfully submitted to OTTO – Multi-Objective Recommender System